# Web Scraping no site da UEA

O objetivo deste scraping é coletar os dados referentes às grades curriculares dos cursos da UEA e reorganizá-los em arquivos _csv_.

Deixo como referência principal o tutorial de scraping do [RodrigoCMoraes](1), que além de ser simples, também referencia outros bons tutoriais. Recomendo!

[1]:https://github.com/RodrigoCMoraes/web_scraping/blob/master/download_editais_fapeam/Download%20de%20Editais%20Vigentes%20FAPEAM.ipynb

<h1>Tabela de Conteúdos<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Primeira-página-de-login" data-toc-modified-id="Primeira-página-de-login-1">Primeira página de login</a></span></li><li><span><a href="#Segunda-página-de-login-(???)" data-toc-modified-id="Segunda-página-de-login-(???)-2">Segunda página de login (???)</a></span></li><li><span><a href="#Página-com-informação-sobre-os-cursos" data-toc-modified-id="Página-com-informação-sobre-os-cursos-3">Página com informação sobre os cursos</a></span></li><li><span><a href="#Grade-curricular" data-toc-modified-id="Grade-curricular-4">Grade curricular</a></span></li><li><span><a href="#Tratamento-dos-dados" data-toc-modified-id="Tratamento-dos-dados-5">Tratamento dos dados</a></span></li></ul></div>

In [ ]:
import re, sys, requests, json
from bs4 import BeautifulSoup, Tag
from ast import literal_eval
import pandas as pd
sys.setrecursionlimit(10000)

In [ ]:
# Funções auxiliares

def fix(s):
    if s is None:
        s = ''
    return re.sub(' \w\w? ', ',', s)

def is_grade_curricular(tag):
    return tag.name == 'a' and tag.next_element == 'Grade Curricular'

# mostra o histórico dos redirecionamentos
def track(last_page):
    for page in last_page.history+[last_page]:
        print(f'{page}: {page.url} (text_len={len(page.text)})')

In [ ]:
session = requests.Session()
# Melhorar forma de esconder as credenciais?
config = json.load(open('credentials.json'))

### Primeira página de login

Para saber qual as chaves do payload, basta inspecionar o elemento com o mouse em cima da caixa de login.

In [ ]:
payload_login = dict(
    username=config['login']['username'], # atribuindo dessa forma pra não ocultar as chaves ;)
    password=config['login']['password'])
url_login = 'http://www1.uea.edu.br/modulo/login/lyceum2.php'
result = session.post(url_login, data=payload_login, headers=dict(referer=url_login))
track(result)

### Segunda página de login (???)

Aqui as coisas ficam um pouco estranhas (ou eu realmente não tenho conhecimento sobre essa "técnica").

Basicamente, se você analisar a aba _networking_ do _inspecionar elemento_ enquanto faz o login no aluno online, o site rapidamente o redireciona para outra página de login, antes de realizar o login de fato. Essa seguna página de login é preenchida automaticamente, mas o que me preocupa é que para logar no aluno online, **você não precisa logar na primeira página, apenas na segunda**.

Isso é, observando o payload dessa requisição POST, o necessário para alguém entrar na sua conta é saber sua matrícula, seu *txtsenha_tac* e sua *txtsenha*. Não sei o que são esses dois últimos parâmetros, mas eu espero que seja uma chave gerada utilizando nosso email institucional + senha.

Vale notar também que essas possíveis chaves são constantes, logo, não mudam por sessão. Bom, fica o mistério do porquê é implementado desse jeito (pelo menos para mim).

Segue uma pequena demonstração de como pegar o valor desses parâmetros para _seu_ login:
![](media/middle_logon.gif)

Será que podemos automatizar? :)

In [ ]:
payload_middle_logon = dict(
    txtnumero_matricula=config['middle_logon']['txtnumero_matricula'],
    txtsenha_tac       =config['middle_logon']['txtsenha_tac'],
    txtsenha           =config['middle_logon']['txtsenha'])
url_middle_logon = 'http://www1.uea.edu.br/lyceump/aonline/middle_logon.asp'
result = session.post(url_middle_logon, data=payload_middle_logon, headers=dict(refer=url_middle_logon))
track(result)

### Página com informação sobre os cursos

![cursos](media/cursos.png)

In [ ]:
url_cursos = 'http://www1.uea.edu.br/lyceump/aonline/cursos.asp'
result = session.get(url_cursos)
content = BeautifulSoup(result.content, 'lxml')
track(result)

### Grade curricular

Inicialmente seleciono apenas uma grade de um curso específico, mas é possível generalizar para todos as grades sem muita dificuldade.

![grade](media/grade.png)

In [ ]:
links = content.find_all(is_grade_curricular)
link = links[1874] # grade atual de SI

keys = ['cursonome', 'curso', 'turno', 'curriculo', 'alt']
params_raw = re.search('\((.*)\);$', link['href']).groups(1)[0]
params = list(literal_eval(params_raw))
payload = {key: val for key, val in zip(keys, params)}
print(payload)
url_grade = 'http://www1.uea.edu.br/lyceump/aonline/grade.asp'
result = session.post(url_grade, data=payload, headers=dict(refer=url_grade))
track(result)

### Tratamento dos dados

In [ ]:
# Não usar lxml como parser: https://stackoverflow.com/a/21655159
content = BeautifulSoup(result.content, 'html5lib')

In [ ]:
# Seleciona a primeira linha "amarela" na tabela
# Depois, itera sobre as linhas da tabela e trata as informações relevantes
first_row = content.find(class_='tr01t')
relevant_rows = first_row.find_next_siblings(class_=['font02', 'tr01', 'tr01a'])
periodo = 1
disciplinas = []
for row in relevant_rows:
    if (row['class'] == ['font02']):
        periodo += 1
    else:
        cod_nome, prereqs, equivs = (dado.string for dado in row.children)
        codigo = cod_nome.split()[0]
        nome = " ".join(cod_nome.split()[2:])
        prereqs = fix(prereqs)
        equivs = fix(equivs)
        disciplinas.append([codigo, nome, prereqs, equivs, periodo])

columns = ['cod', 'nome', 'prereq', 'equiv', 'periodo']
grade = pd.DataFrame(disciplinas, columns=columns).set_index('cod')
grade # Precisa de redução transitiva.

In [ ]:
grade.to_csv('data.csv')